This is a page for exploring the STAC collections available for a given site (sub-catalog)

In [ ]:
from operator import attrgetter

import panel as pn
import pystac

import utils.cards
import utils.location
import utils.template
import utils.utils

pn.extension()

In [ ]:
query_params = utils.location.UrlQueryParams()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(query_params, {"site_id": "site-id", "collection_id": "collection-id"})  # type: ignore

In [ ]:
def get_site_card(site: pystac.Catalog | None):
    if site is None:
        return "Site not found"
    else:
        return utils.cards.site(site, collapsed=True)

In [ ]:
def get_collection_cards(site: pystac.Catalog | None) -> list:
    if site is None:
        return ["No collections found"]
    collections = utils.utils.get_collections(site)
    collections = sorted(collections, key=attrgetter("title", "id"))
    return [utils.cards.collection(site, c, with_links=True, collapsed=True) for c in collections]

In [ ]:
def get_content(site: pystac.Catalog | None) -> pn.Column:
    elements = []

    if site is None:
        elements.append("Site not found")
    else:
        site_name = site.title or site.id
        elements.append(
            pn.pane.Markdown(
                f"""
                # Collections

                On this page you can discover the data collections available for the site {site_name}.
                Each collection contains a specific type of data related to peat health,
                such as multi-year timeseries of biogeophysical, hydrological, and surface motion variables.

                Check out one of the collections below to explore the data.
                """
            )
        )
        elements.append(get_site_card(site))
        elements.append(pn.pane.Markdown("## The data collections"))
        elements.extend(get_collection_cards(site))

    return pn.Column(
        *elements,
        align=("start", "center"),
        min_width=400,
        max_width=1200,
        sizing_mode="stretch_width",
    )

In [ ]:
content = pn.bind(
    get_content,
    site=query_params.param.site,
)

In [ ]:
template = utils.template.get_template(content)
template.servable();